In [ ]:
pip install pandas psycopg2

In [ ]:
import pandas as pd
import psycopg2
import time

# Database connection parameters
db_params = {
    "dbname": "demo",
    "user": "postgres",
    "password": "1234qwer",  # replace 'your_password' with your actual password
    "host": "postgres",
    "port": 5432
}

def connect_db(params):
    """Establish a connection to the PostgreSQL database."""
    conn = psycopg2.connect(**params)
    return conn

def insert_data(row, conn):
    """Insert a single row into the PostgreSQL table."""
    with conn.cursor() as cur:
        cur.execute("""
        INSERT INTO bos_air_traffic (icao24, callsign, origin_country, time_position, last_contact, longitude, 
                                     latitude, baro_altitude, on_ground, velocity, true_track, vertical_rate, sensors, 
                                     geo_altitude, squawk, spi, position_source)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (callsign, last_contact) DO NOTHING;
        """, (row['icao24'], row['callsign'], row['origin_country'], row['time_position'], row['last_contact'],
              row['longitude'], row['latitude'], row['baro_altitude'], row['on_ground'], row['velocity'],
              row['true_track'], row['vertical_rate'], row['sensors'], row['geo_altitude'], row['squawk'],
              row['spi'], row['position_source']))
        conn.commit()

def process_data(file_path, db_params):
    """Read data from CSV and insert into the database."""
    # Connect to the database
    conn = connect_db(db_params)
    
    try:
        # Load data from CSV
        df = pd.read_csv(file_path)
        
        # Iterate over the DataFrame rows
        for index, row in df.iterrows():
            insert_data(row, conn)
            print(f"Inserted row {index + 1}")
            time.sleep(3)  # Wait for 3 seconds before the next insert
    finally:
        conn.close()  # Ensure the connection is closed

if __name__ == "__main__":
    file_path = 'us_boston_air_traffic_hourly.csv'  # Path to the CSV file
    process_data(file_path, db_params)
